## Demo Workshop

This is a demo script. This script captures creation of a new conference on OpenReview system.

Following are the steps and brief explanations:


### Login as an Administrator

In [1]:
import openreview
import icml

client = openreview.Client(baseurl='https://dev.openreview.net',username=<Admin username>,password=<Admin password>)

### Create a Group object for the new conference and post this Group object

A hierarchical structure is recommended for conferences. 
As an example, to create a conference for ICML 201,we start by creating a root group for this conference with "id" = "ICML.cc". 
We, then, proceed to making subgroups with id = "ICML.cc/2019" and "ICML.cc/2019/Conference".

Note that the web field for the conference is a JS file.

In [12]:
print client.post_group(openreview.Group(id = 'ICML.cc', 
                                         readers = ['everyone'], 
                                         writers = ['OpenReview.net'], 
                                         signatories = ['ICML.cc'], 
                                         signatures = ['OpenReview.net']))

{'cdate': 1531336263548,
 'ddate': None,
 'id': u'ICML.cc',
 'members': [],
 'nonreaders': [],
 'readers': [u'everyone'],
 'signatories': [u'ICML.cc'],
 'signatures': [u'OpenReview.net'],
 'web': None,
 'writers': [u'OpenReview.net']}


In [13]:
print client.post_group(openreview.Group(id = 'ICML.cc/2019', 
                                         readers = ['everyone'], 
                                         writers = ['OpenReview.net'], 
                                         signatories = ['ICML.cc/2019'], 
                                         signatures = ['OpenReview.net']))

{'cdate': 1531336337516,
 'ddate': None,
 'id': u'ICML.cc/2019',
 'members': [],
 'nonreaders': [],
 'readers': [u'everyone'],
 'signatories': [u'ICML.cc/2019'],
 'signatures': [u'OpenReview.net'],
 'web': None,
 'writers': [u'OpenReview.net']}


In [18]:
print client.post_group(openreview.Group(id = 'ICML.cc/2019/Conference', 
                                         readers = ['everyone'], 
                                         writers = ['OpenReview.net'], 
                                         signatories = ['ICML.cc/2019/Conference'], 
                                         signatures = ['OpenReview.net'],
                                         web = <path to file containing the js code for the webfield>))

{'cdate': 1531339441440,
 'ddate': None,
 'id': u'ICML.cc/2019/Conference',
 'members': [],
 'nonreaders': [],
 'readers': [u'everyone'],
 'signatories': [u'ICML.cc/2019/Conference'],
 'signatures': [u'OpenReview.net'],
 'web': u'\n// ------------------------------------\n// Basic venue homepage template\n//\n// This webfield displays the conference header (#header), the submit button (#invitation),\n// and a list of all submitted papers (#notes).\n// ------------------------------------\n\n// Constants\nvar CONFERENCE = "ICML.cc/2019/Conference";\nvar INVITATION = CONFERENCE + \'/-/Submission\';\nvar SUBJECT_AREAS = [\n  // Add conference specific subject areas here\n];\nvar BUFFER = 1000 * 60 * 30;  // 30 minutes\nvar PAGE_SIZE = 50;\n\nvar paperDisplayOptions = {\n  pdfLink: true,\n  replyCount: true,\n  showContents: true\n};\n\n// Main is the entry point to the webfield code and runs everything\nfunction main() {\n  Webfield.ui.setup(\'#group-container\', CONFERENCE);  // required

### Create a Submission Invitation for this conference

In [17]:
print client.post_invitation(openreview.Invitation(id = 'ICML.cc/2019/Conference/-/Submission', 
                                            readers = ['everyone'],
                                            writers = ['ICML.cc/2019/Conference'],
                                            signatures = ['ICML.cc/2019/Conference'],
                                            invitees = ['everyone'],
                                            duedate = 1562875092000,
                                            reply = {
                                                    'forum': None,
                                                    'replyto': None,
                                                    'readers': {
                                                        'description': 'The users who will be allowed to read the above content.',
                                                        'values': ['everyone']
                                                    },
                                                    'signatures': {
                                                        'description': 'Your authorized identity to be associated with the above content.',
                                                        'values-regex': '~.*'
                                                    },
                                                    'writers': {
                                                        'values-regex': '~.*'
                                                    },
                                                    'content':{
                                                        'title': {
                                                            'description': 'Title of paper.',
                                                            'order': 1,
                                                            'value-regex': '.{1,250}',
                                                            'required':True
                                                        },
                                                        'authors': {
                                                            'description': 'Comma separated list of author names. Please provide real names; identities will be anonymized.',
                                                            'order': 2,
                                                            'values-regex': "[^;,\\n]+(,[^,\\n]+)*",
                                                            'required':True
                                                        },
                                                        'authorids': {
                                                            'description': 'Comma separated list of author email addresses, lowercased, in the same order as above. For authors with existing OpenReview accounts, please make sure that the provided email address(es) match those listed in the author\'s profile. Please provide real emails; identities will be anonymized.',
                                                            'order': 3,
                                                            'values-regex': "([a-z0-9_\-\.]{2,}@[a-z0-9_\-\.]{2,}\.[a-z]{2,},){0,}([a-z0-9_\-\.]{2,}@[a-z0-9_\-\.]{2,}\.[a-z]{2,})",
                                                            'required':True
                                                        },
                                                        'abstract': {
                                                            'description': 'Abstract of paper.',
                                                            'order': 4,
                                                            'value-regex': '[\\S\\s]{1,5000}',
                                                            'required':True
                                                        },
                                                        'pdf': {
                                                            'description': 'Upload a PDF file that ends with .pdf',
                                                            'order': 5,
                                                            'value-regex': 'upload',
                                                            'required':True
                                                        }
                                                    }

                                            }))

{'cdate': 1531339106644,
 'ddate': None,
 'duedate': 1562875092000,
 'id': u'ICML.cc/2019/Conference/-/Submission',
 'invitees': [u'everyone'],
 'multiReply': None,
 'noninvitees': [],
 'nonreaders': [],
 'process': None,
 'rdate': None,
 'readers': [u'everyone'],
 'reply': {u'content': {u'abstract': {u'description': u'Abstract of paper.',
                                      u'order': 4,
                                      u'required': True,
                                      u'value-regex': u'[\\S\\s]{1,5000}'},
                        u'authorids': {u'description': u"Comma separated list of author email addresses, lowercased, in the same order as above. For authors with existing OpenReview accounts, please make sure that the provided email address(es) match those listed in the author's profile. Please provide real emails; identities will be anonymized.",
                                       u'order': 3,
                                       u'required': True,
              

### Create a Comment Invitation using this submission invitation

In [23]:
print client.post_invitation(openreview.Invitation(id = 'ICML.cc/2019/Conference/-/Comment',
                                                   readers = ['everyone'],
                                                   writers = ['ICML.cc/2019/Conference'],
                                                   signatures = ['ICML.cc/2019/Conference'],
                                                   invitees = ['everyone'],
                                                    reply = {
                                                            'invitation': 'ICML.cc/2019/Conference/-/Submission',
                                                            'content': {
                                                                'title': {
                                                                    'description': 'Comment title',
                                                                    'order': 1,
                                                                    'value-regex': '.*'
                                                                },
                                                                'comment': {
                                                                    'description': 'Comment',
                                                                    'order': 2,
                                                                    'value-regex': '.{0,1000}'
                                                                }
                                                            },
                                                            'readers': {
                                                                'values': ['everyone']
                                                            },
                                                            'signatures': {
                                                                'values-regex': '\\(anonymous\\)|~.*'
                                                            },
                                                            'writers': {
                                                                'values-regex': '\\(anonymous\\)|~.*'
                                                            }
                                                    }))

{'cdate': 1531340152826,
 'ddate': None,
 'duedate': None,
 'id': u'ICML.cc/2019/Conference/-/Comment',
 'invitees': [u'everyone'],
 'multiReply': None,
 'noninvitees': [],
 'nonreaders': [],
 'process': None,
 'rdate': None,
 'readers': [u'everyone'],
 'reply': {u'content': {u'comment': {u'description': u'Comment',
                                     u'order': 2,
                                     u'value-regex': u'.{0,1000}'},
                        u'title': {u'description': u'Comment title',
                                   u'order': 1,
                                   u'value-regex': u'.*'}},
           u'invitation': u'ICML.cc/2019/Conference/-/Submission',
           u'readers': {u'values': [u'everyone']},
           u'signatures': {u'values-regex': u'\\(anonymous\\)|~.*'},
           u'writers': {u'values-regex': u'\\(anonymous\\)|~.*'}},
 'signatures': [u'ICML.cc/2019/Conference'],
 'taskCompletionCount': None,
 'transform': None,
 'web': None,
 'writers': [u'ICML.cc/2